In [3]:
import numpy as np
import pandas as pd
import pickle

In [4]:
import xgboost

In [5]:
with open ("/Users/nanlin/msds/Adv_ML/leanplum/data/LeanPlum/session.pkl", "rb") as f:
    sessions = pickle.load(f)

/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: 'pandas.core' is private. Use 'pandas.Categorical'
  


In [6]:
sessions.head()

,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,user_id_hash
0,5558845121177764917,1542215364580,Asia/Manila,28800000.0,25837591,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,18781111175537580,1539215568666,Asia/Manila,28800000.0,11343848,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,1477540082628742048,1540120743010,Asia/Manila,28800000.0,13499724,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,8184875317380844086,1542671625528,Asia/Manila,28800000.0,32788010,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,4706180700083856343,1538997913013,Asia/Manila,28800000.0,5872534,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,4,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [35]:
with open ("/Users/nanlin/msds/Adv_ML/leanplum/data/LeanPlum/events.pkl", "rb") as f:
    events = pickle.load(f)

In [36]:
events.head()

,session_id,event,event_timestamp,event_value,user_id_hash
0,5558845121177764917,45,1542215397132,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,5558845121177764917,45,1542215484895,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,7689508378645584666,.m5100869650219008,1541124410372,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,2201961907282901522,4,1543713091129,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,2201961907282901522,6,1543713093116,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [37]:
df1 = sessions[["user_id_hash", "previous_sessions_duration"]]

In [38]:
# events
df2 = events[["user_id_hash", "event_timestamp", "event", "event_value"]]

In [39]:
df1.head()

,user_id_hash,previous_sessions_duration
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,25837591
1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,11343848
2,9943447915df3a45fd6720a026af905b6da6b56a37701b...,13499724
3,9943447915df3a45fd6720a026af905b6da6b56a37701b...,32788010
4,9943447915df3a45fd6720a026af905b6da6b56a37701b...,5872534


In [40]:
df2.head()

,user_id_hash,event_timestamp,event,event_value
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1542215397132,45,0.0
1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1542215484895,45,0.0
2,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1541124410372,.m5100869650219008,0.0
3,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1543713091129,4,0.0
4,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1543713093116,6,0.0


In [41]:
# get unique user list
users = pd.DataFrame(list(events.user_id_hash.unique()))

In [42]:
users.columns = ["user_id_hash"]

#### Use Dec-1 to Dec-7 and Dec-1 to Dec-14 to get labels for the validation set

In [43]:
dec1 = df2[(df2["event_timestamp"] >= 1543651199000) & (df2["event_timestamp"] < 1544255999000)]

In [44]:
puser1 = set(dec1[dec1["event"] == "8"].user_id_hash.unique())

In [45]:
users_val = users

In [46]:
labels1 = []
for user in list(events.user_id_hash.unique()):
    if user in puser1:
        labels1.append(1)
    else:
        labels1.append(0)

In [47]:
labels1[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [48]:
users_val["purchased1"] = pd.DataFrame(labels1)

In [49]:
users_val.head()

,user_id_hash,purchased1
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0


In [50]:
dec2 = df2[(df2["event_timestamp"] >= 1544255999000)]

In [51]:
puser2 = set(dec2[dec2["event"] == "8"].user_id_hash.unique())

In [52]:
labels2 = []
for user in list(events.user_id_hash.unique()):
    if user in puser2:
        labels2.append(1)
    else:
        labels2.append(0)

In [53]:
users_val["purchased2"] = pd.DataFrame(labels2)

In [54]:
users_val.head()

,user_id_hash,purchased1,purchased2
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0


#### Use Oct-15 to Nov-30 to get features for users labeled in December

In [56]:
nov = df2[(df2["event_timestamp"] >= 1539586800000) & (df2["event_timestamp"] < 1543651199000)]

In [57]:
nov.head()

,user_id_hash,event_timestamp,event,event_value
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1542215397132,45,0.0
1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1542215484895,45,0.0
2,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1541124410372,.m5100869650219008,0.0
118,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1541320475697,45,0.0
119,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1541320589748,45,0.0


##### Feature 1: amount purchased by user

In [58]:
def purchaseAmt(df):
    df = df[df["event"]=="8"][["user_id_hash", "event_value"]].groupby("user_id_hash").sum().reset_index()
    return df

In [59]:
feature1 = nov[["user_id_hash", "event", "event_value"]]

In [60]:
feature1.head()

,user_id_hash,event,event_value
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,45,0.0
1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,45,0.0
2,9943447915df3a45fd6720a026af905b6da6b56a37701b...,.m5100869650219008,0.0
118,9943447915df3a45fd6720a026af905b6da6b56a37701b...,45,0.0
119,9943447915df3a45fd6720a026af905b6da6b56a37701b...,45,0.0


In [61]:
feature1 = feature1[feature1["event"]=="8"][["user_id_hash", "event_value"]].groupby("user_id_hash").sum()

In [62]:
feature1 = feature1.reset_index()

In [63]:
feature1.head()

,user_id_hash,event_value
0,002e447aed33ed4c51a68743cc293ef2148058b6a6239e...,0.0
1,005c9d79cf18efc6c8b5fa767964b1def9b8e2f8abd23f...,0.0
2,0061b4d30e8a9935b2ebeec954ff509b4f0cb500cd725c...,0.0
3,00fbbf507c7d3c2f259cd3329d241c29d35712e2d4699f...,0.0
4,0107a0017873efc2dea9a9155832363ceacf6fe97bd428...,0.0


In [64]:
users_val = users_val.join(feature1.set_index("user_id_hash"), on="user_id_hash", how="left")

In [65]:
users_val.head()

,user_id_hash,purchased1,purchased2,event_value
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0,3.492188
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0,0.000000
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0,0.000000
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0,0.000000
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0,0.000000


In [66]:
users_val.count()

user_id_hash    620988
purchased1      620988
purchased2      620988
event_value     620988
dtype: int64

In [67]:
users_val.columns = ["user_id_hash", "purchase1", "purchased2", "amt_purchased"]

##### feature 2: distinct number of purchases made by user

In [27]:
def distinctPurchase(df):
    df = df[df["event"]=="8"].groupby("user_id_hash").count().reset_index()
    return df

In [28]:
feature2 = nov[["user_id_hash", "event"]]

In [29]:
feature2 = feature2[feature2["event"]=="8"].groupby("user_id_hash").count()

In [30]:
feature2 = feature2.reset_index()

In [41]:
feature2.head()

,user_id_hash,event
0,002e447aed33ed4c51a68743cc293ef2148058b6a6239e...,0
1,005c9d79cf18efc6c8b5fa767964b1def9b8e2f8abd23f...,0
2,0061b4d30e8a9935b2ebeec954ff509b4f0cb500cd725c...,0
3,00fbbf507c7d3c2f259cd3329d241c29d35712e2d4699f...,0
4,0107a0017873efc2dea9a9155832363ceacf6fe97bd428...,0


In [42]:
feature2.describe()

,event
count,621001.000000
mean,0.211467
std,2.111923
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,205.000000


In [31]:
users_val = users_val.join(feature2.set_index("user_id_hash"), on="user_id_hash", how="left")

In [44]:
users_val.count()

user_id_hash     620988
purchase1        620988
purchased2       620988
amt_purchased    620988
event            620988
dtype: int64

In [45]:
users_val.head()

,user_id_hash,purchase1,purchased2,amt_purchased,event
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0,3.492188,1
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0,0.000000,0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0,0.000000,0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0,0.000000,0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0,0.000000,0


##### Feature 3: total previous session duration

In [46]:
sessions.head()

,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,user_id_hash
0,5558845121177764917,1542215364580,Asia/Manila,28800000.0,25837591,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,18781111175537580,1539215568666,Asia/Manila,28800000.0,11343848,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,1477540082628742048,1540120743010,Asia/Manila,28800000.0,13499724,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,8184875317380844086,1542671625528,Asia/Manila,28800000.0,32788010,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,4706180700083856343,1538997913013,Asia/Manila,28800000.0,5872534,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,4,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [32]:
tmp = sessions[["user_id_hash", "previous_sessions_duration", "start_timestamp"]]

In [48]:
tmp.head()

,user_id_hash,previous_sessions_duration,start_timestamp
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,25837591,1542215364580
1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,11343848,1539215568666
2,9943447915df3a45fd6720a026af905b6da6b56a37701b...,13499724,1540120743010
3,9943447915df3a45fd6720a026af905b6da6b56a37701b...,32788010,1542671625528
4,9943447915df3a45fd6720a026af905b6da6b56a37701b...,5872534,1538997913013


In [33]:
tmp = tmp[(tmp["start_timestamp"] >= 1539586800000) & (tmp["start_timestamp"] < 1543651199000)]

In [50]:
tmp.head()

,user_id_hash,previous_sessions_duration,start_timestamp
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,25837591,1542215364580
3,9943447915df3a45fd6720a026af905b6da6b56a37701b...,32788010,1542671625528
7,9943447915df3a45fd6720a026af905b6da6b56a37701b...,14113619,1541320437287
8,9943447915df3a45fd6720a026af905b6da6b56a37701b...,32827242,1542674647158
9,9943447915df3a45fd6720a026af905b6da6b56a37701b...,28049619,1542496808264


In [34]:
feature3 = tmp[["user_id_hash", "previous_sessions_duration"]].groupby("user_id_hash").sum()

In [35]:
feature3 = feature3.reset_index()
feature3.head()

,user_id_hash,previous_sessions_duration
0,0013ae7e9f73194aeb0d350f1de1f441625825efe403c4...,4346056.0
1,0025a6c398e0ec7c471342188141853877f94a76b51270...,43615420.0
2,002e3b6443aef897fc35486e072c5781d087943cdb9dff...,0.0
3,002e447aed33ed4c51a68743cc293ef2148058b6a6239e...,7557137.0
4,005c9d79cf18efc6c8b5fa767964b1def9b8e2f8abd23f...,0.0


In [36]:
users_val = users_val.join(feature3.set_index("user_id_hash"), on="user_id_hash", how="left")

In [37]:
users_val.head()

,user_id_hash,purchase1,purchased2,amt_purchased,event,previous_sessions_duration
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0,3.492188,1,857156805.0
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0,0.000000,0,6074816.0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0,0.000000,0,0.0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0,0.000000,0,0.0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0,0.000000,0,0.0


In [38]:
users_val.columns = ["user_id_hash", "purchase1", "purchase2", "amt_purchased", \
                     "num_of_purchase", "total_prev_session_duration"]

In [39]:
users_val.head()

,user_id_hash,purchase1,purchase2,amt_purchased,num_of_purchase,total_prev_session_duration
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0,3.492188,1,857156805.0
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0,0.000000,0,6074816.0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0,0.000000,0,0.0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0,0.000000,0,0.0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0,0.000000,0,0.0


In [57]:
users_val.to_csv("users_val.csv", index=False)

In [40]:
users_val.describe()

,purchase1,purchase2,amt_purchased,num_of_purchase,total_prev_session_duration
count,620988.000000,620988.000000,620988.0,620988.000000,6.209880e+05
mean,0.007383,0.004979,NaN,0.322644,2.461968e+08
std,0.085609,0.070387,NaN,2.788809,2.796797e+09
min,0.000000,0.000000,0.0,0.000000,0.000000e+00
25%,0.000000,0.000000,0.0,0.000000,0.000000e+00
50%,0.000000,0.000000,0.0,0.000000,0.000000e+00
75%,0.000000,0.000000,0.0,0.000000,6.770604e+06
max,1.000000,1.000000,1054.0,445.000000,6.370117e+11


##### Feature 4: total count of different events in event

In [134]:
events.head()

,session_id,event,event_timestamp,event_value,user_id_hash
0,5558845121177764917,45,1542215397132,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,5558845121177764917,45,1542215484895,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,7689508378645584666,.m5100869650219008,1541124410372,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,2201961907282901522,4,1543713091129,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,2201961907282901522,6,1543713093116,0.0,9943447915df3a45fd6720a026af905b6da6b56a37701b...


In [41]:
tmp2 = events[["user_id_hash", "event", "event_value", "event_timestamp"]]

In [198]:
tmp2.head()

,user_id_hash,event,event_value,event_timestamp
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,45,0.0,1542215397132
1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,45,0.0,1542215484895
2,9943447915df3a45fd6720a026af905b6da6b56a37701b...,.m5100869650219008,0.0,1541124410372
3,9943447915df3a45fd6720a026af905b6da6b56a37701b...,4,0.0,1543713091129
4,9943447915df3a45fd6720a026af905b6da6b56a37701b...,6,0.0,1543713093116


In [42]:
tmp3 = tmp2[tmp2["event_value"]!=0]

In [43]:
tmp3.event.unique()

[5, 7, 11, 8, 9, 10]
Categories (6, object): [5, 7, 11, 8, 9, 10]

In [202]:
tmp3.head()

,user_id_hash,event,event_value,event_timestamp
5,9943447915df3a45fd6720a026af905b6da6b56a37701b...,5,1.0,1543713101700
25,9943447915df3a45fd6720a026af905b6da6b56a37701b...,5,1.0,1543714449189
32,9943447915df3a45fd6720a026af905b6da6b56a37701b...,5,1.0,1539215610646
67,9943447915df3a45fd6720a026af905b6da6b56a37701b...,7,12.0,1538875188252
74,9943447915df3a45fd6720a026af905b6da6b56a37701b...,7,20.0,1538875448149


In [44]:
# count event=5
tmp3 = tmp3[tmp3["event"]=="5"][["user_id_hash", "event"]].groupby("user_id_hash").count().reset_index()

In [44]:
tmp3.head()

,user_id_hash,event
0,002e447aed33ed4c51a68743cc293ef2148058b6a6239e...,4
1,005c9d79cf18efc6c8b5fa767964b1def9b8e2f8abd23f...,0
2,0061b4d30e8a9935b2ebeec954ff509b4f0cb500cd725c...,0
3,00fbbf507c7d3c2f259cd3329d241c29d35712e2d4699f...,1
4,0107a0017873efc2dea9a9155832363ceacf6fe97bd428...,0


In [45]:
feature4 = tmp3

In [46]:
users_val = users_val.join(feature4.set_index("user_id_hash"), on="user_id_hash", how="left")

In [208]:
users_val.head()

,user_id_hash,purchase1,purchase2,amt_purchased,num_of_purchase,total_prev_session_duration,event
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0,3.492188,1,817125640.0,28
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0,0.000000,0,6074816.0,2
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0,0.000000,0,0.0,0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0,0.000000,0,0.0,0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0,0.000000,0,0.0,0


In [47]:
users_val.columns = ["user_id_hash", "purchase1", "purchase2", "amt_purchased", "num_of_purchase", \
                    "total_prev_session_duration", "num_of_event5"]

In [48]:
def distinctEvent(df, value):
    df = df[df["event"]==value].groupby("user_id_hash").count().reset_index()
    return df

In [49]:
feature5 = distinctEvent(tmp2[["user_id_hash", "event"]], "7")

In [50]:
feature5.columns = ["user_id_hash", "num_of_event7"]

In [51]:
users_val = users_val.join(feature5.set_index("user_id_hash"), on="user_id_hash", how="left")

In [52]:
users_val.head()

,user_id_hash,purchase1,purchase2,amt_purchased,num_of_purchase,total_prev_session_duration,num_of_event5,num_of_event7
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0,3.492188,1,817125640.0,28,9
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0,0.000000,0,6074816.0,2,2
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0,0.000000,0,0.0,0,3
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0,0.000000,0,0.0,0,0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0,0.000000,0,0.0,0,0


In [52]:
feature6 = distinctEvent(tmp2[["user_id_hash", "event"]], "9")
feature6.columns = ["user_id_hash", "num_of_event9"]
users_val = users_val.join(feature6.set_index("user_id_hash"), on="user_id_hash", how="left")

In [53]:
users_val.columns = ["user_id_hash", "purchase1", "purchase2", "amt_purchased", "num_of_purchase", \
                    "total_prev_session_duration", "num_of_event5","num_of_event7", "num_of_event9"]

In [54]:
feature7 = distinctEvent(tmp2[["user_id_hash", "event"]], "10")
feature7.columns = ["user_id_hash", "num_of_event10"]
users_val = users_val.join(feature7.set_index("user_id_hash"), on="user_id_hash", how="left")

In [55]:
users_val.columns = ["user_id_hash", "purchase1", "purchase2", "amt_purchased", "num_of_purchase", \
                    "total_prev_session_duration", "num_of_event5","num_of_event7", "num_of_event9", "num_of_event10"]

In [56]:
feature8 = distinctEvent(tmp2[["user_id_hash", "event"]], "11")
feature8.columns = ["user_id_hash", "num_of_event11"]
users_val = users_val.join(feature8.set_index("user_id_hash"), on="user_id_hash", how="left")

In [57]:
def eventAmt(df, value):
    df = df[df["event"]==value][["user_id_hash", "event_value"]].groupby("user_id_hash").sum().reset_index()
    return df

In [58]:
# feature 9
feature9 = eventAmt(df2[["user_id_hash", "event_value", "event"]], "5")
feature9.columns = ["user_id_hash", "amt_of_event5"]
users_val = users_val.join(feature9.set_index("user_id_hash"), on="user_id_hash", how="left")

In [59]:
feature10 = eventAmt(df2[["user_id_hash", "event_value", "event"]], "7")
feature10.columns = ["user_id_hash", "amt_of_event7"]
users_val = users_val.join(feature10.set_index("user_id_hash"), on="user_id_hash", how="left")

In [60]:
feature11 = eventAmt(df2[["user_id_hash", "event_value", "event"]], "9")
feature11.columns = ["user_id_hash", "amt_of_event9"]
users_val = users_val.join(feature11.set_index("user_id_hash"), on="user_id_hash", how="left")

In [61]:
feature12 = eventAmt(df2[["user_id_hash", "event_value", "event"]], "10")
feature12.columns = ["user_id_hash", "amt_of_event10"]
users_val = users_val.join(feature12.set_index("user_id_hash"), on="user_id_hash", how="left")

In [62]:
feature13 = eventAmt(df2[["user_id_hash", "event_value", "event"]], "11")
feature13.columns = ["user_id_hash", "amt_of_event11"]
users_val = users_val.join(feature13.set_index("user_id_hash"), on="user_id_hash", how="left")

In [63]:
users_val.head()

,user_id_hash,purchase1,purchase2,amt_purchased,num_of_purchase,total_prev_session_duration,num_of_event5,num_of_event7,num_of_event9,num_of_event10,num_of_event11,amt_of_event5,amt_of_event7,amt_of_event9,amt_of_event10,amt_of_event11
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0,3.492188,1,857156805.0,28,9,0,0,1,28.0,167.0,0.0,0.0,5.0
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0,0.000000,0,6074816.0,2,2,0,0,1,2.0,45.0,0.0,0.0,5.0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0,0.000000,0,0.0,0,3,0,0,0,0.0,47.0,0.0,0.0,0.0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0,0.000000,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0,0.000000,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0


In [347]:
users_val.describe()

,purchase1,purchase2,amt_purchased,num_of_purchase,total_prev_session_duration,num_of_event5,num_of_event7,num_of_event9,num_of_event10,num_of_event11,amt_of_event5,amt_of_event7,amt_of_event9,amt_of_event10,amt_of_event11
count,620988.000000,620988.000000,620988.0,620988.000000,6.209880e+05,620988.000000,620988.000000,620988.000000,620988.000000,620988.000000,620988.0,620988.0,620988.0,620988.0,620988.0
mean,0.007383,0.004979,NaN,0.211471,1.888030e+08,5.705094,3.173818,0.138218,0.003804,0.110031,NaN,NaN,NaN,NaN,NaN
std,0.085609,0.070387,NaN,2.111945,2.285150e+09,18.352764,12.581608,0.699218,0.067591,0.316062,NaN,NaN,NaN,NaN,0.0
min,0.000000,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
25%,0.000000,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
50%,0.000000,0.000000,0.0,0.000000,0.000000e+00,0.000000,2.000000,0.000000,0.000000,0.000000,0.0,27.0,0.0,0.0,0.0
75%,0.000000,0.000000,0.0,0.000000,1.978582e+06,3.000000,3.000000,0.000000,0.000000,0.000000,3.0,58.0,0.0,0.0,0.0
max,1.000000,1.000000,997.0,205.000000,5.160220e+11,777.000000,1579.000000,110.000000,5.000000,6.000000,inf,inf,inf,inf,30.0


In [104]:
users_val[users_val["amt_of_event5"]==float("inf")] = 0
users_val[users_val["amt_of_event7"]==float("inf")] = 0
users_val[users_val["amt_of_event9"]==float("inf")] = 0
users_val[users_val["amt_of_event10"]==float("inf")] = 0
users_val[users_val["amt_of_event11"]==float("inf")] = 0

In [105]:
users_val["amt_purchased"].fillna(0, inplace=True)
users_val["num_of_purchase"].fillna(0, inplace=True)
users_val["total_prev_session_duration"].fillna(0, inplace=True)
users_val["num_of_event5"].fillna(0, inplace=True)
users_val["num_of_event7"].fillna(0, inplace=True)
users_val["num_of_event9"].fillna(0, inplace=True)
users_val["num_of_event10"].fillna(0, inplace=True)
users_val["num_of_event11"].fillna(0, inplace=True)
users_val["amt_of_event5"].fillna(0, inplace=True)
users_val["amt_of_event7"].fillna(0, inplace=True)
users_val["amt_of_event9"].fillna(0, inplace=True)
users_val["amt_of_event10"].fillna(0, inplace=True)
users_val["amt_of_event11"].fillna(0, inplace=True)

In [106]:
X_val = users_val[["amt_purchased", "num_of_purchase", "total_prev_session_duration", "num_of_event5", \
                       "num_of_event7", "num_of_event9", "num_of_event10", "num_of_event11",\
                      "amt_of_event5", "amt_of_event7", "amt_of_event9", "amt_of_event10", "amt_of_event11"]]
y_val_1 = users_val[["purchase1"]]
y_val_2 = users_val[["purchase2"]]

## feature 5: count day of user

In [69]:
sessions.head()

,session_id,start_timestamp,timezone,timezone_offset,previous_sessions_duration,user_created_timestamp,is_user_first_session,country,region,city,latitude,longitude,locale,os_name,session_index,device_id,user_id_hash
0,5558845121177764917,1542215364580,Asia/Manila,28800000.0,25837591,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,30,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
1,18781111175537580,1539215568666,Asia/Manila,28800000.0,11343848,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,10,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
2,1477540082628742048,1540120743010,Asia/Manila,28800000.0,13499724,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,13,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
3,8184875317380844086,1542671625528,Asia/Manila,28800000.0,32788010,1538874289458,False,PH,00,makati,14.554729,121.024445,en_GB,Android OS,41,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...
4,4706180700083856343,1538997913013,Asia/Manila,28800000.0,5872534,1538874289458,False,PH,11,davao city,7.190708,125.455338,en_GB,Android OS,4,546a3d98-d540-4e72-ad82-9ebd64e0839b,9943447915df3a45fd6720a026af905b6da6b56a37701b...


#### Using Nov-15 to Nov-22 and Nov-22 to Nov-28 to get training labels

In [68]:
nov1 = df2[(df2["event_timestamp"] >= 1542268800000) & (df2["event_timestamp"] < 1542960000000)]

puser1 = set(nov1[nov1["event"] == "8"].user_id_hash.unique())

users_train = users

labels1 = []
for user in list(events.user_id_hash.unique()):
    if user in puser1:
        labels1.append(1)
    else:
        labels1.append(0)

labels1[:10]

users_train["purchased1"] = pd.DataFrame(labels1)

In [69]:
nov2 = df2[(df2["event_timestamp"] >= 1542268800000) & (df2["event_timestamp"] < 1543478400000)]

puser2 = set(nov2[nov2["event"] == "8"].user_id_hash.unique())

labels2 = []
for user in list(events.user_id_hash.unique()):
    if user in puser2:
        labels2.append(1)
    else:
        labels2.append(0)

users_train["purchased2"] = pd.DataFrame(labels2)

users_train.head()

,user_id_hash,purchased1,purchased2
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0


#### Use data before Nov-15 to get features

In [70]:
october = df2[(df2["event_timestamp"] < 1542268800000)]
october.head()

,user_id_hash,event_timestamp,event,event_value
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1542215397132,45,0.0
1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1542215484895,45,0.0
2,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1541124410372,.m5100869650219008,0.0
30,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1539215572790,.m6335456823869440,0.0
31,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1539215608399,1,0.0


In [71]:
# feature 1
feature1_train = purchaseAmt(october[["user_id_hash", "event", "event_value"]])
feature1_train.head()

,user_id_hash,event_value
0,002e447aed33ed4c51a68743cc293ef2148058b6a6239e...,0.0
1,005c9d79cf18efc6c8b5fa767964b1def9b8e2f8abd23f...,0.0
2,0061b4d30e8a9935b2ebeec954ff509b4f0cb500cd725c...,0.0
3,00fbbf507c7d3c2f259cd3329d241c29d35712e2d4699f...,0.0
4,0107a0017873efc2dea9a9155832363ceacf6fe97bd428...,0.0


In [72]:
users_train = users_train.join(feature1_train.set_index("user_id_hash"), on="user_id_hash", how="left")

In [73]:
users_train.columns = ["user_id_hash", "purchased1", "purchased2", "amt_purchased"]

In [74]:
users_train.head()

,user_id_hash,purchased1,purchased2,amt_purchased
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0,3.492188
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0,0.000000
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0,0.000000
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0,0.000000
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0,0.000000


In [75]:
# feature 2
feature2_train = distinctPurchase(october[["user_id_hash", "event"]])
feature2_train.head()

,user_id_hash,event
0,002e447aed33ed4c51a68743cc293ef2148058b6a6239e...,0
1,005c9d79cf18efc6c8b5fa767964b1def9b8e2f8abd23f...,0
2,0061b4d30e8a9935b2ebeec954ff509b4f0cb500cd725c...,0
3,00fbbf507c7d3c2f259cd3329d241c29d35712e2d4699f...,0
4,0107a0017873efc2dea9a9155832363ceacf6fe97bd428...,0


In [76]:
users_train = users_train.join(feature2_train.set_index("user_id_hash"), on="user_id_hash", how="left")

In [77]:
users_train.columns = ["user_id_hash", "purchased1", "purchased2", "amt_purchased", "num_of_purchase"]

In [78]:
users_train.head()

,user_id_hash,purchased1,purchased2,amt_purchased,num_of_purchase
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0,3.492188,1
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0,0.000000,0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0,0.000000,0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0,0.000000,0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0,0.000000,0


In [79]:
# feature 3
tmp = sessions[["user_id_hash", "previous_sessions_duration", "start_timestamp"]]
tmp = tmp[(tmp["start_timestamp"] < 1542268800000)]
feature3_train = tmp[["user_id_hash", "previous_sessions_duration"]].groupby("user_id_hash").sum()
feature3_train = feature3_train.reset_index()
users_train = users_train.join(feature3_train.set_index("user_id_hash"), on="user_id_hash", how="left")
users_train.columns = ["user_id_hash", "purchase1", "purchase2", "amt_purchased", \
                     "num_of_purchase", "total_prev_session_duration"]
users_train.head()

,user_id_hash,purchase1,purchase2,amt_purchased,num_of_purchase,total_prev_session_duration
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,0,0,3.492188,1,471711092.0
1,43f75f8042d3c80c45e222bdd09267f4584684c54d6fae...,0,0,0.000000,0,6074816.0
2,999524249720812f2d8c0390293efd58e1ac84d587a01c...,0,0,0.000000,0,0.0
3,4e6bc35cf7fd79a5312047651e7865915f4a6bec193cf2...,0,0,0.000000,0,0.0
4,dc009148ee26d658e0240c7b7f6a258790a457737f96e8...,0,0,0.000000,0,0.0


In [80]:
train_features = october[["user_id_hash", "event", "event_value", "event_timestamp"]]
train_features.head()

,user_id_hash,event,event_value,event_timestamp
0,9943447915df3a45fd6720a026af905b6da6b56a37701b...,45,0.0,1542215397132
1,9943447915df3a45fd6720a026af905b6da6b56a37701b...,45,0.0,1542215484895
2,9943447915df3a45fd6720a026af905b6da6b56a37701b...,.m5100869650219008,0.0,1541124410372
30,9943447915df3a45fd6720a026af905b6da6b56a37701b...,.m6335456823869440,0.0,1539215572790
31,9943447915df3a45fd6720a026af905b6da6b56a37701b...,1,0.0,1539215608399


In [81]:
feature5_train = distinctEvent(train_features[["user_id_hash", "event"]], "5")
feature5_train.columns = ["user_id_hash", "num_of_event5"]
users_train = users_train.join(feature5_train.set_index("user_id_hash"), on="user_id_hash", how="left")

In [82]:
feature7_train = distinctEvent(train_features[["user_id_hash", "event"]], "7")
feature7_train.columns = ["user_id_hash", "num_of_event7"]
users_train = users_train.join(feature7_train.set_index("user_id_hash"), on="user_id_hash", how="left")

In [83]:
feature9_train = distinctEvent(train_features[["user_id_hash", "event"]], "9")
feature9_train.columns = ["user_id_hash", "num_of_event9"]
users_train = users_train.join(feature9_train.set_index("user_id_hash"), on="user_id_hash", how="left")

In [84]:
feature10_train = distinctEvent(train_features[["user_id_hash", "event"]], "10")
feature10_train.columns = ["user_id_hash", "num_of_event10"]
users_train = users_train.join(feature10_train.set_index("user_id_hash"), on="user_id_hash", how="left")

In [85]:
feature11_train = distinctEvent(train_features[["user_id_hash", "event"]], "11")
feature11_train.columns = ["user_id_hash", "num_of_event11"]
users_train = users_train.join(feature11_train.set_index("user_id_hash"), on="user_id_hash", how="left")

In [86]:
feature12_train = eventAmt(train_features[["user_id_hash", "event_value", "event"]], "5")
feature12_train.columns = ["user_id_hash", "amt_of_event5"]
users_train = users_train.join(feature12_train.set_index("user_id_hash"), on="user_id_hash", how="left")

In [87]:
feature13_train = eventAmt(train_features[["user_id_hash", "event_value", "event"]], "7")
feature13_train.columns = ["user_id_hash", "amt_of_event7"]
users_train = users_train.join(feature13_train.set_index("user_id_hash"), on="user_id_hash", how="left")

In [88]:
feature14_train = eventAmt(train_features[["user_id_hash", "event_value", "event"]], "9")
feature14_train.columns = ["user_id_hash", "amt_of_event9"]
users_train = users_train.join(feature14_train.set_index("user_id_hash"), on="user_id_hash", how="left")

In [89]:
feature15_train = eventAmt(train_features[["user_id_hash", "event_value", "event"]], "10")
feature15_train.columns = ["user_id_hash", "amt_of_event10"]
users_train = users_train.join(feature15_train.set_index("user_id_hash"), on="user_id_hash", how="left")

In [90]:
feature16_train = eventAmt(train_features[["user_id_hash", "event_value", "event"]], "11")
feature16_train.columns = ["user_id_hash", "amt_of_event11"]
users_train = users_train.join(feature16_train.set_index("user_id_hash"), on="user_id_hash", how="left")

In [91]:
users_train[users_train["amt_of_event5"]==float("inf")] = 0

In [92]:
users_train[users_train["amt_of_event7"]==float("inf")] = 0

In [93]:
users_train[users_train["amt_of_event9"]==float("inf")] = 0

In [94]:
users_train[users_train["amt_of_event10"]==float("inf")] = 0

In [95]:
users_train.describe()

,purchase1,purchase2,amt_purchased,num_of_purchase,total_prev_session_duration,num_of_event5,num_of_event7,num_of_event9,num_of_event10,num_of_event11,amt_of_event5,amt_of_event7,amt_of_event9,amt_of_event10,amt_of_event11
count,620988.000000,620988.000000,620988.0,620988.000000,6.209880e+05,620988.000000,620988.000000,620988.000000,620988.000000,620988.000000,620988.0,620988.0,620988.0,620988.0,620988.0
mean,0.012372,0.018818,NaN,0.266522,1.476713e+08,4.915942,2.100202,0.090230,0.002705,0.080731,NaN,NaN,0.0,0.0,NaN
std,0.110540,0.135883,NaN,2.477481,1.707691e+09,13.970800,8.252592,0.496252,0.056715,0.275016,NaN,NaN,0.0,NaN,0.0
min,0.000000,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
25%,0.000000,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
50%,0.000000,0.000000,0.0,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
75%,0.000000,0.000000,0.0,0.000000,2.868456e+06,3.000000,3.000000,0.000000,0.000000,0.000000,2.0,52.0,0.0,0.0,0.0
max,1.000000,1.000000,826.0,444.000000,4.742721e+11,604.000000,1027.000000,60.000000,5.000000,6.000000,560.0,13008.0,60.0,1950.0,30.0


In [96]:
users_train.count()

user_id_hash                   620988
purchase1                      620988
purchase2                      620988
amt_purchased                  620988
num_of_purchase                620988
total_prev_session_duration    620988
num_of_event5                  620988
num_of_event7                  620988
num_of_event9                  620988
num_of_event10                 620988
num_of_event11                 620988
amt_of_event5                  620988
amt_of_event7                  620988
amt_of_event9                  620988
amt_of_event10                 620988
amt_of_event11                 620988
dtype: int64

In [107]:
users_train["amt_purchased"].fillna(0, inplace=True)
users_train["num_of_purchase"].fillna(0, inplace=True)
users_train["total_prev_session_duration"].fillna(0, inplace=True)
users_train["num_of_event5"].fillna(0, inplace=True)
users_train["num_of_event7"].fillna(0, inplace=True)
users_train["num_of_event9"].fillna(0, inplace=True)
users_train["num_of_event10"].fillna(0, inplace=True)
users_train["num_of_event11"].fillna(0, inplace=True)
users_train["amt_of_event5"].fillna(0, inplace=True)
users_train["amt_of_event7"].fillna(0, inplace=True)
users_train["amt_of_event9"].fillna(0, inplace=True)
users_train["amt_of_event10"].fillna(0, inplace=True)
users_train["amt_of_event11"].fillna(0, inplace=True)

In [108]:
X_train = users_train[["amt_purchased", "num_of_purchase", "total_prev_session_duration", "num_of_event5", \
                       "num_of_event7", "num_of_event9", "num_of_event10", "num_of_event11",\
                      "amt_of_event5", "amt_of_event7", "amt_of_event9", "amt_of_event10", "amt_of_event11"]]
y_train_1 = users_train[["purchase1"]]
y_train_2 = users_train[["purchase2"]]

In [109]:
X_train.describe()

,amt_purchased,num_of_purchase,total_prev_session_duration,num_of_event5,num_of_event7,num_of_event9,num_of_event10,num_of_event11,amt_of_event5,amt_of_event7,amt_of_event9,amt_of_event10,amt_of_event11
count,620988.0,620988.000000,6.209880e+05,620988.000000,620988.000000,620988.000000,620988.000000,620988.000000,620988.0,620988.0,620988.0,620988.0,620988.0
mean,NaN,0.266522,1.476713e+08,4.915942,2.100202,0.090230,0.002705,0.080731,NaN,NaN,0.0,0.0,NaN
std,NaN,2.477481,1.707691e+09,13.970800,8.252592,0.496252,0.056715,0.275016,NaN,NaN,0.0,NaN,0.0
min,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.000000,2.868456e+06,3.000000,3.000000,0.000000,0.000000,0.000000,2.0,52.0,0.0,0.0,0.0
max,826.0,444.000000,4.742721e+11,604.000000,1027.000000,60.000000,5.000000,6.000000,560.0,13008.0,60.0,1950.0,30.0


#### Model Training

In [98]:
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics

In [205]:
rf = RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_leaf=50, max_features=5, verbose=1)

In [206]:
rf.fit(X_train, y_train_1)

/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   25.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  7.8min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  9.6min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=50, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [207]:
rf.feature_importances_

array([2.37887949e-01, 2.86829317e-01, 8.14205286e-02, 7.23285002e-02,
       7.70338082e-02, 4.98469217e-03, 3.58145299e-06, 3.93574769e-03,
       6.91657050e-02, 1.53823071e-01, 5.04695147e-03, 3.90333377e-06,
       7.53624477e-03])

In [208]:
pred_train=rf.predict_proba(X_train)
pred_val1 = rf.predict_proba(X_val)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    5.7s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   12.7s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:   21.8s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:   27.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    5.2s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   11.6s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:   20.4s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:   25.7s finished


In [209]:
# Training AUC 7 day 
sklearn.metrics.roc_auc_score(y_train_1, pred_train[:,-1])

0.8870712526538618

In [210]:
# Validation AUC 7 day 
sklearn.metrics.roc_auc_score(y_val_1, pred_val1[:,-1])

0.951670385815893

In [113]:
rf2 = RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_leaf=50, max_features=5, verbose=1)
rf2.fit(X_train, y_train_2)
pred_train2 = rf.predict_proba(X_train)
pred_val2 = rf.predict_proba(X_val)

/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  9.1min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    7.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   16.0s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:   33.6s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 

In [114]:
rf2.feature_importances_

array([2.39172241e-01, 2.64090139e-01, 7.61888361e-02, 1.09146279e-01,
       8.02599894e-02, 4.43765561e-03, 7.72960917e-06, 3.15124762e-03,
       6.70038868e-02, 1.45475406e-01, 4.60605327e-03, 6.95549300e-06,
       6.45358162e-03])

In [115]:
print(f"AUC For Training set 14 days purchase is: {sklearn.metrics.roc_auc_score(y_train_2, pred_train2[:,-1])}")

AUC For Training set 14 days purchase is: 0.8591736726263093


In [116]:
print(f"AUC For Val set 14 days purchase is: {sklearn.metrics.roc_auc_score(y_val_2, pred_val2[:,-1])}")

AUC For Val set 14 days purchase is: 0.9500080526797494


In [ ]:
from sklearn.model_selection import GridSearchCV

##### XGBoost

In [145]:
import xgboost as xgb
from datetime import timedelta
import datetime as dt

In [146]:
from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [147]:
train_dmatrix1 = xgb.DMatrix(data=X_train,label=y_train_1)
train_dmatrix2 = xgb.DMatrix(data=X_train,label=y_train_2)
val_dmatrix1 = xgb.DMatrix(data=X_val,label=y_val_1)
val_dmatrix2 = xgb.DMatrix(data=X_val,label=y_val_2)

In [155]:
clf = XGBClassifier(learning_rate = 0.01, max_depth = 3, n_estimators = 500, n_jobs=-1, \
                    colsample_bytree=1, gamma=1, subsample=0.8)

In [156]:
clf.fit(X_train, y_train_1)

/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)

In [157]:
pred_train_xgb = clf.predict_proba(X_train)
pred_val1_xgb = clf.predict_proba(X_val)

In [158]:
clf.feature_importances_

array([0.17697595, 0.08591066, 0.21821305, 0.19329897, 0.0083047 ,
       0.00200458, 0.00429553, 0.00744559, 0.19043528, 0.10538373,
       0.        , 0.        , 0.00773196], dtype=float32)

In [159]:
print(f"AUC For Training set 7 days purchase is: {sklearn.metrics.roc_auc_score(y_train_1, pred_train_xgb[:,-1])}")

AUC For Training set 7 days purchase is: 0.8421454804803913


In [160]:
print(f"AUC For Val set 7 days purchase is: {sklearn.metrics.roc_auc_score(y_val_1, pred_val1_xgb[:,-1])}")

AUC For Val set 7 days purchase is: 0.8708249566916312


In [161]:
# 14 days purchase
clf2 = XGBClassifier(learning_rate = 0.01, max_depth = 3, n_estimators = 500, n_jobs=-1, \
                    colsample_bytree=1, gamma=1, subsample=0.8)
clf2.fit(X_train, y_train_2)

/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)

In [162]:
pred_train2_xgb = clf2.predict_proba(X_train)
pred_val2_xgb = clf2.predict_proba(X_val)

In [163]:
print(f"AUC For Training set 14 days purchase is: {sklearn.metrics.roc_auc_score(y_train_2, pred_train2_xgb[:,-1])}")
print(f"AUC For Training set 14 days purchase is: {sklearn.metrics.roc_auc_score(y_val_2, pred_val2_xgb[:,-1])}")

AUC For Training set 14 days purchase is: 0.833297996471884
AUC For Training set 14 days purchase is: 0.8635433134592677


Random Tuning Hyperparameters

In [211]:
clf = XGBClassifier(learning_rate = 0.05, max_depth = 10, n_estimators = 500, n_jobs=-1, \
                    colsample_bytree=0.6, gamma=1, subsample=0.8)
clf.fit(X_train, y_train_1)
pred_train_xgb = clf.predict_proba(X_train)
pred_val1_xgb = clf.predict_proba(X_val)

/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [212]:
print(f"AUC For Training set 7 days purchase is: {sklearn.metrics.roc_auc_score(y_train_1, pred_train_xgb[:,-1])}")
print(f"AUC For Val set 7 days purchase is: {sklearn.metrics.roc_auc_score(y_val_1, pred_val1_xgb[:,-1])}")

AUC For Training set 7 days purchase is: 0.877319837989774
AUC For Val set 7 days purchase is: 0.9216035539046179


In [213]:
# 14 days purchase
clf2 = XGBClassifier(learning_rate = 0.05, max_depth = 10, n_estimators = 500, n_jobs=-1, \
                    colsample_bytree=0.6, gamma=1, subsample=0.8)
clf2.fit(X_train, y_train_2)

pred_train2_xgb = clf2.predict_proba(X_train)
pred_val2_xgb = clf2.predict_proba(X_val)

print(f"AUC For Training set 14 days purchase is: {sklearn.metrics.roc_auc_score(y_train_2, pred_train2_xgb[:,-1])}")
print(f"AUC For Training set 14 days purchase is: {sklearn.metrics.roc_auc_score(y_val_2, pred_val2_xgb[:,-1])}")

/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AUC For Training set 14 days purchase is: 0.8554503530064015
AUC For Training set 14 days purchase is: 0.9024786789805543


In [218]:
clf.feature_importances_

array([0.0958171 , 0.0655936 , 0.24459709, 0.12103752, 0.08939922,
       0.03506254, 0.00102522, 0.01584991, 0.11029321, 0.1786344 ,
       0.01490671, 0.00036908, 0.02741439], dtype=float32)

In [219]:
clf2.feature_importances_

array([0.0946563 , 0.06313091, 0.24278961, 0.12303717, 0.09002964,
       0.03591171, 0.00114164, 0.01588287, 0.10841612, 0.18178177,
       0.01560247, 0.00042061, 0.02719917], dtype=float32)

In [167]:
clf = XGBClassifier(learning_rate = 0.05, max_depth = 10, n_estimators = 1000, n_jobs=-1, \
                    colsample_bytree=0.5, gamma=1, subsample=0.8)
clf.fit(X_train, y_train_1)
pred_train_xgb = clf.predict_proba(X_train)
pred_val1_xgb = clf.predict_proba(X_val)

print(f"AUC For Training set 7 days purchase is: {sklearn.metrics.roc_auc_score(y_train_1, pred_train_xgb[:,-1])}")
print(f"AUC For Val set 7 days purchase is: {sklearn.metrics.roc_auc_score(y_val_1, pred_val1_xgb[:,-1])}")

/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AUC For Training set 7 days purchase is: 0.8865725330292712
AUC For Val set 7 days purchase is: 0.885915965527597


In [168]:
# 14 days purchase
clf2 = XGBClassifier(learning_rate = 0.05, max_depth = 10, n_estimators = 1000, n_jobs=-1, \
                    colsample_bytree=0.5, gamma=1, subsample=0.8)
clf2.fit(X_train, y_train_2)

pred_train2_xgb = clf2.predict_proba(X_train)
pred_val2_xgb = clf2.predict_proba(X_val)

print(f"AUC For Training set 14 days purchase is: {sklearn.metrics.roc_auc_score(y_train_2, pred_train2_xgb[:,-1])}")
print(f"AUC For Training set 14 days purchase is: {sklearn.metrics.roc_auc_score(y_val_2, pred_val2_xgb[:,-1])}")

/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/envs/ml/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AUC For Training set 14 days purchase is: 0.8640035590327249
AUC For Training set 14 days purchase is: 0.8842745661002024


#### Compute Targets

In [169]:
# getting users
sample = pd.read_csv("/Users/zhangxinran/msds630_adv_ML/leanplum/sample_submission_2.csv")
sample.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.01,0.02
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.01,0.02
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.01,0.02
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.01,0.02
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.01,0.02


In [171]:
users_test = sample[["user_id_hash"]]

##### Getting target feature using data from Nov-1 to Dec-14

In [172]:
test_features = df2[(df2["event_timestamp"] >= 1541055600000)]
test_features.head()

# feature 1
test_feature1 = purchaseAmt(test_features[["user_id_hash", "event", "event_value"]])
test_feature1.head()

,user_id_hash,event_value
0,002e447aed33ed4c51a68743cc293ef2148058b6a6239e...,0.0
1,005c9d79cf18efc6c8b5fa767964b1def9b8e2f8abd23f...,0.0
2,0061b4d30e8a9935b2ebeec954ff509b4f0cb500cd725c...,0.0
3,00fbbf507c7d3c2f259cd3329d241c29d35712e2d4699f...,0.0
4,0107a0017873efc2dea9a9155832363ceacf6fe97bd428...,0.0


In [173]:
users_test = users_test.join(test_feature1.set_index("user_id_hash"), on="user_id_hash", how="left")
users_test.columns = ["user_id_hash", "amt_purchased"]
# feature 2
test_feature2 = distinctPurchase(test_features[["user_id_hash", "event"]])
users_test = users_test.join(test_feature2.set_index("user_id_hash"), on="user_id_hash", how="left")
users_test.columns = ["user_id_hash", "amt_purchased", "num_of_purchase"]

In [174]:
users_test.head()

,user_id_hash,amt_purchased,num_of_purchase
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.0,0.0
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.0,0.0
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.0,0.0
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.0,0.0
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.0,0.0


In [175]:
# feature 3
tmp = sessions[["user_id_hash", "previous_sessions_duration", "start_timestamp"]]
tmp = tmp[(tmp["start_timestamp"] >= 1542268801000)]
test_feature3 = tmp[["user_id_hash", "previous_sessions_duration"]].groupby("user_id_hash").sum()
test_feature3 = test_feature3.reset_index()
users_test = users_test.join(test_feature3.set_index("user_id_hash"), on="user_id_hash", how="left")
users_test.columns = ["user_id_hash", "amt_purchased", "num_of_purchase", "total_prev_session_duration"]

In [176]:
# feature 4 - 9
feature5_test = distinctEvent(test_features[["user_id_hash", "event"]], "5")
feature5_test.columns = ["user_id_hash", "num_of_event5"]
users_test = users_test.join(feature5_test.set_index("user_id_hash"), on="user_id_hash", how="left")

In [177]:
feature7_test = distinctEvent(test_features[["user_id_hash", "event"]], "7")
feature7_test.columns = ["user_id_hash", "num_of_event7"]
users_test = users_test.join(feature7_test.set_index("user_id_hash"), on="user_id_hash", how="left")

In [178]:
feature9_test = distinctEvent(test_features[["user_id_hash", "event"]], "9")
feature9_test.columns = ["user_id_hash", "num_of_event9"]
users_test = users_test.join(feature9_test.set_index("user_id_hash"), on="user_id_hash", how="left")

In [179]:
feature10_test = distinctEvent(test_features[["user_id_hash", "event"]], "10")
feature10_test.columns = ["user_id_hash", "num_of_event10"]
users_test = users_test.join(feature10_test.set_index("user_id_hash"), on="user_id_hash", how="left")

In [180]:
feature11_test = distinctEvent(test_features[["user_id_hash", "event"]], "11")
feature11_test.columns = ["user_id_hash", "num_of_event11"]
users_test = users_test.join(feature11_test.set_index("user_id_hash"), on="user_id_hash", how="left")

In [181]:
feature12_test = eventAmt(test_features[["user_id_hash", "event_value", "event"]], "5")
feature12_test.columns = ["user_id_hash", "amt_of_event5"]
users_test = users_test.join(feature12_test.set_index("user_id_hash"), on="user_id_hash", how="left")

In [182]:
feature13_test = eventAmt(test_features[["user_id_hash", "event_value", "event"]], "7")
feature13_test.columns = ["user_id_hash", "amt_of_event7"]
users_test = users_test.join(feature13_test.set_index("user_id_hash"), on="user_id_hash", how="left")

In [183]:
feature14_test = eventAmt(test_features[["user_id_hash", "event_value", "event"]], "9")
feature14_test.columns = ["user_id_hash", "amt_of_event9"]
users_test = users_test.join(feature14_test.set_index("user_id_hash"), on="user_id_hash", how="left")

In [184]:
feature15_test = eventAmt(test_features[["user_id_hash", "event_value", "event"]], "10")
feature15_test.columns = ["user_id_hash", "amt_of_event10"]
users_test = users_test.join(feature15_test.set_index("user_id_hash"), on="user_id_hash", how="left")

In [185]:
feature16_test = eventAmt(test_features[["user_id_hash", "event_value", "event"]], "11")
feature16_test.columns = ["user_id_hash", "amt_of_event11"]
users_test = users_test.join(feature16_test.set_index("user_id_hash"), on="user_id_hash", how="left")

In [186]:
users_test.head()

,user_id_hash,amt_purchased,num_of_purchase,total_prev_session_duration,num_of_event5,num_of_event7,num_of_event9,num_of_event10,num_of_event11,amt_of_event5,amt_of_event7,amt_of_event9,amt_of_event10,amt_of_event11
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,2.0,17.0,0.0,0.0,0.0
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,17.0,0.0,0.0,0.0
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [187]:
users_test[users_test["amt_of_event5"]==float("inf")] = 0
users_test[users_test["amt_of_event7"]==float("inf")] = 0
users_test[users_test["amt_of_event9"]==float("inf")] = 0
users_test[users_test["amt_of_event10"]==float("inf")] = 0
users_test[users_test["amt_of_event11"]==float("inf")] = 0

In [188]:
users_test.describe()

,amt_purchased,num_of_purchase,total_prev_session_duration,num_of_event5,num_of_event7,num_of_event9,num_of_event10,num_of_event11,amt_of_event5,amt_of_event7,amt_of_event9,amt_of_event10,amt_of_event11
count,312316.0,312316.000000,3.123680e+05,312316.000000,312316.000000,312316.000000,312316.000000,312316.000000,312316.0,312316.0,312316.0,312316.0,312316.0
mean,NaN,0.324521,2.413255e+08,5.033661,2.044087,0.084764,0.001972,0.056894,NaN,NaN,0.0,0.0,NaN
std,NaN,3.053174,3.018820e+09,17.130488,11.637891,0.569848,0.047907,0.233897,NaN,NaN,0.0,0.0,0.0
min,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.000000,0.000000e+00,3.000000,2.000000,0.000000,0.000000,0.000000,1.0,32.0,0.0,0.0,0.0
max,1135.0,236.000000,4.960723e+11,820.000000,1540.000000,87.000000,3.000000,4.000000,674.0,19440.0,87.0,90.0,20.0


In [190]:
users_test.count()

user_id_hash                   312568
amt_purchased                  312316
num_of_purchase                312316
total_prev_session_duration    312368
num_of_event5                  312316
num_of_event7                  312316
num_of_event9                  312316
num_of_event10                 312316
num_of_event11                 312316
amt_of_event5                  312316
amt_of_event7                  312316
amt_of_event9                  312316
amt_of_event10                 312316
amt_of_event11                 312316
dtype: int64

In [191]:
users_test["amt_purchased"].fillna(0, inplace=True)
users_test["num_of_purchase"].fillna(0, inplace=True)
users_test["total_prev_session_duration"].fillna(0, inplace=True)
users_test["num_of_event5"].fillna(0, inplace=True)
users_test["num_of_event7"].fillna(0, inplace=True)
users_test["num_of_event9"].fillna(0, inplace=True)
users_test["num_of_event10"].fillna(0, inplace=True)
users_test["num_of_event11"].fillna(0, inplace=True)
users_test["amt_of_event5"].fillna(0, inplace=True)
users_test["amt_of_event7"].fillna(0, inplace=True)
users_test["amt_of_event9"].fillna(0, inplace=True)
users_test["amt_of_event10"].fillna(0, inplace=True)
users_test["amt_of_event11"].fillna(0, inplace=True)

In [192]:
users_test.count()

user_id_hash                   312568
amt_purchased                  312568
num_of_purchase                312568
total_prev_session_duration    312568
num_of_event5                  312568
num_of_event7                  312568
num_of_event9                  312568
num_of_event10                 312568
num_of_event11                 312568
amt_of_event5                  312568
amt_of_event7                  312568
amt_of_event9                  312568
amt_of_event10                 312568
amt_of_event11                 312568
dtype: int64

In [193]:
X_test = users_test[["amt_purchased", "num_of_purchase", "total_prev_session_duration", "num_of_event5", \
                       "num_of_event7", "num_of_event9", "num_of_event10", "num_of_event11",\
                      "amt_of_event5", "amt_of_event7", "amt_of_event9", "amt_of_event10", "amt_of_event11"]]

In [194]:
pred_1 = rf.predict_proba(X_test)
pred_2 = rf2.predict_proba(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.7s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    8.2s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:   13.9s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:   16.5s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    2.6s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:   11.9s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:   14.6s finished


In [195]:
pred_1

array([[9.84861514e-01, 1.51384857e-02],
       [9.98009572e-01, 1.99042790e-03],
       [9.98848114e-01, 1.15188636e-03],
       ...,
       [9.99377125e-01, 6.22874628e-04],
       [9.84861514e-01, 1.51384857e-02],
       [9.84861514e-01, 1.51384857e-02]])

In [196]:
sample["user_purchase_binary_7_days"] = pd.DataFrame(pred_1[:,-1])

In [197]:
sample["user_purchase_binary_14_days"] = pd.DataFrame(pred_2[:,-1])

In [198]:
sample.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.015138,0.029678
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.001990,0.001558
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.001152,0.002654
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.000470,0.000307
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.015138,0.029678


In [199]:
submission = pd.read_csv("/Users/zhangxinran/msds630_adv_ML/leanplum/sample_submission_2.csv")

In [200]:
s = sample[["user_purchase_binary_7_days"]].astype('double', copy=False)

In [201]:
s2 = sample[["user_purchase_binary_14_days"]].astype('double', copy=False)

In [202]:
submission.loc[:, "user_purchase_binary_7_days"] = s
submission.loc[:, "user_purchase_binary_14_days"] = s2

In [203]:
submission.to_csv("~/Desktop/submission5.csv", index=False)

In [204]:
submission.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.015138,0.029678
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.001990,0.001558
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.001152,0.002654
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.000470,0.000307
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.015138,0.029678


In [ ]:
submission.describe()

In [214]:
pred_1_xgb = clf.predict_proba(X_test)
pred_2_xgb = clf2.predict_proba(X_test)

In [215]:
sample["user_purchase_binary_7_days"] = pd.DataFrame(pred_1_xgb[:,-1])
sample["user_purchase_binary_14_days"] = pd.DataFrame(pred_2_xgb[:,-1])

In [216]:
submission = pd.read_csv("/Users/zhangxinran/msds630_adv_ML/leanplum/sample_submission_2.csv")
s = sample[["user_purchase_binary_7_days"]].astype('double', copy=False)
s2 = sample[["user_purchase_binary_14_days"]].astype('double', copy=False)
submission.loc[:, "user_purchase_binary_7_days"] = s
submission.loc[:, "user_purchase_binary_14_days"] = s2
submission.to_csv("~/Desktop/submission6.csv", index=False)

In [217]:
submission.head()

,user_id_hash,user_purchase_binary_7_days,user_purchase_binary_14_days
0,e469dfaed039ead9110165d9bc457acb11609ca34057dc...,0.015373,0.030254
1,afcc639a324b6c598ef83d360450afa011cb2dd1358bf9...,0.003408,0.002802
2,fd5a7cf211d08e3e00f7be6a9df6e6ea3d2e5c22a5d9c3...,0.001838,0.002779
3,00bfff98b9d0329f014c2eeac7ce47cd18b2bc6e10d608...,0.001475,0.001443
4,0d298f3638c43e915c119d4935e1ce8d168f81b5e3e8c1...,0.015373,0.030254
